In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
def calculate_vortex(df, value, n=14):
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [4]:
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

## volatility

In [10]:

# Flatten MultiIndex columns 
xly.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in xly.columns
]

# Calculate True Range
xly["prev_close"] = xly["Close_XLY"].shift(1)
xly["tr1"] = xly["High_XLY"] - xly["Low_XLY"]
xly["tr2"] = abs(xly["High_XLY"] - xly["prev_close"])
xly["tr3"] = abs(xly["Low_XLY"] - xly["prev_close"])

xly["true_range"] = xly[["tr1", "tr2", "tr3"]].max(axis=1)

# 10-day ATR
xly["ATR_10"] = xly["true_range"].rolling(window=10).mean()

# ---- STEP 4: Calculate ATR as a percentage of closing price ----
xly["atr_pct"] = xly["ATR_10"] / xly["Close_XLY"]

# allocating the capital

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

xly["position_size"] = xly.apply(position_size, axis=1)

# ---- STEP 6: Optional - Capital allocation per trade ----
#capital = 100000 # Example: $100K total portfolio
#xly["allocation_dollars"] = xly["position_size"] * capital

# ---- Preview ----
print(xly[["Close_XLY", "ATR_10", "atr_pct", "position_size"]].tail(10))


             Close_XLY    ATR_10   atr_pct  position_size
Date                                                     
2025-02-19  225.618988  2.870099  0.012721           0.01
2025-02-20  223.674316  2.919964  0.013055           0.01
2025-02-21  217.790527  3.453495  0.015857           0.01
2025-02-24  216.972778  3.270997  0.015076           0.01
2025-02-25  215.835892  3.511334  0.016269           0.01
2025-02-26  214.948349  3.602083  0.016758           0.01
2025-02-27  211.846878  3.751672  0.017709           0.01
2025-02-28  215.367203  3.836439  0.017813           0.01
2025-03-03  211.398117  4.429805  0.020955           0.01
2025-03-04  207.668396  4.845659  0.023334           0.01


In [11]:
import plotly.express as px
fig = px.line(xly, x=xly.index, y="atr_pct", title="ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [12]:
import plotly.express as px

# Filter only 2025 data
xly_2025 = xly[xly.index.year == 2025]

# Plot
fig = px.line(xly_2025, x=xly_2025.index, y="atr_pct", title="ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [21]:
# Without sentiment score
xly_copy = xly.copy()
xly_copy['atr_pct'] = xly_copy['ATR_10'] / xly_copy['Close_XLY']

# Create Buy Signal (assuming VI_Cross_Up is defined elsewhere)
xly_copy['Buy_Signal'] = xly_copy['VI+_'] > xly_copy['VI-_']  # Vortex crossover
# + add any other buy conditions here...

# Create Sell Signal (basic)
xly_copy['Sell_Signal'] = xly_copy['VI-_'] > xly_copy['VI+_']

# Initialize position state
xly_copy['Position'] = 0
peak_price = 0

for i in range(1, len(xly_copy)):
    if xly_copy['Buy_Signal'].iloc[i]:
        xly_copy.at[xly_copy.index[i], 'Position'] = 1
        peak_price = xly_copy['Close_XLY'].iloc[i]
    elif xly_copy['Position'].iloc[i - 1] == 1:
        current_price = xly_copy['Close_XLY'].iloc[i]
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03:
            xly_copy.at[xly_copy.index[i], 'Sell_Signal'] = True  # trailing stop
            xly_copy.at[xly_copy.index[i], 'Position'] = 0
        else:
            xly_copy.at[xly_copy.index[i], 'Position'] = 1


In [22]:
capital = 100000
in_position = False
entry_price = 0
position_value = 0
cash = capital
returns = []

for i in range(len(xly_copy)):
    row = xly_copy.iloc[i]
    
    # Buy
    if row['Buy_Signal'] and not in_position:
        position_size = row['position_size']
        position_value = cash * position_size
        entry_price = row['Close_XLY']
        shares_bought = position_value / entry_price
        cash -= position_value
        in_position = True
        
    # Sell
    elif row['Sell_Signal'] and in_position:
        exit_price = row['Close_XLY']
        proceeds = shares_bought * exit_price
        profit = proceeds - position_value
        cash += proceeds
        returns.append(profit)
        in_position = False
        position_value = 0
        entry_price = 0

# Final capital
final_value = cash + (shares_bought * row['Close_XLY'] if in_position else 0)
total_return = final_value - capital

print(f"Final Capital: ${final_value:.2f}")
print(f"Total Return: ${total_return:.2f}")
print(f"Total Trades: {len(returns)}")
print(f"Average Profit per Trade: ${np.mean(returns):.2f}")


Final Capital: $100732.95
Total Return: $732.95
Total Trades: 75
Average Profit per Trade: $9.77


In [74]:
import vectorbt as vbt

xly = xly_copy.dropna(subset=['Close_XLY'])
entries = xly_copy['Buy_Signal'].astype(bool)
exits = xly_copy['Sell_Signal'].astype(bool)

price = xly_copy['Close_XLY']
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

print(portfolio.stats())
portfolio.plot().show()



/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                              100000.0
End Value                           171782.856932
Total Return [%]                        71.782857
Benchmark Return [%]                   120.815486
Max Gross Exposure [%]                      100.0
Total Fees Paid                      21444.054746
Max Drawdown [%]                        33.668413
Max Drawdown Duration                       793.0
Total Trades                                   75
Total Closed Trades                            75
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            45.333333
Best Trade [%]                          37.025757
Worst Trade [%]                        -13.070507
Avg Winning Trade [%]                    4.635497
Avg Losing Trade [%]                    -2.212751
